In [1]:
import cPickle as pickle
import numpy as np
f = open('/home/mohammad/img_label.p','rb')
label=pickle.load(f)
f.close()

f = open('/home/mohammad/img_data.p','rb')
data=pickle.load(f)
f.close()

In [2]:
label.shape, data.shape

((22424,), (22424, 50, 50))

In [3]:
from sklearn.model_selection import train_test_split
X_train1, X_test, y_train1, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train1, y_train1, test_size=0.2, random_state=42)
print X_train.shape,X_valid.shape, X_test.shape,y_test.shape,y_train.shape

(12019, 50, 50) (3005, 50, 50) (7400, 50, 50) (7400,) (12019,)


In [4]:
def run_cross_validation(nfolds=10):
    # input image dimensions
    img_rows, img_cols = 24, 32
    batch_size = 64
    nb_classes = 10
    nb_epoch = 1
    # number of convolutional filters to use
    nb_filters = 32
    # size of pooling area for max pooling
    nb_pool = 2
    # convolution kernel size
    nb_conv = 3
    random_state = 51

    train_data, train_target = read_and_normalize_train_data(img_rows, img_cols)
    test_data, test_id = read_and_normalize_test_data(img_rows, img_cols)

    yfull_train = dict()
    yfull_test = []
    kf = KFold(len(train_data), n_folds=nfolds, shuffle=True, random_state=random_state)
    num_fold = 0
    for train_index, test_index in kf:
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        X_train, X_valid = train_data[train_index], train_data[test_index]
        Y_train, Y_valid = train_target[train_index], train_target[test_index]
        print('Split train: ', len(X_train))
        print('Split valid: ', len(X_valid))

        model = Sequential()
        model.add(Convolution2D(nb_filters, nb_conv, nb_conv,
                                border_mode='valid',
                                input_shape=(1, img_rows, img_cols)))
        model.add(Activation('relu'))
        model.add(Convolution2D(nb_filters, nb_conv, nb_conv))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(nb_classes))
        model.add(Activation('softmax'))

        model.compile(loss='categorical_crossentropy', optimizer='adadelta')

        model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
                  show_accuracy=True, verbose=1, validation_data=(X_valid, Y_valid))

        # score = model.evaluate(X_valid, Y_valid, show_accuracy=True, verbose=0)
        # print('Score log_loss: ', score[0])

        predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
        score = log_loss(Y_valid, predictions_valid)
        print('Score log_loss: ', score)

        # Store valid predictions
        for i in range(len(test_index)):
            yfull_train[test_index[i]] = predictions_valid[i]

        # Store test predictions
        test_prediction = model.predict(test_data, batch_size=128, verbose=1)
        yfull_test.append(test_prediction)

    score = log_loss(train_target, dict_to_list(yfull_train))
    print('Final score log_loss: ', score)

    test_res = merge_several_folds_fast(yfull_test, nfolds)
    create_submission(test_res, test_id, score)


In [4]:
import tensorflow as tf
import sys

In [5]:
sys.path.append('/home/mohammad/anaconda/lib/python2.7/site-packages')
import keras
from keras.utils import np_utils

Using TensorFlow backend.


In [6]:
image_size=50
num_labels=10
def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  #labels = labels.reshape((-1,10))
  return dataset, labels
X_train,y_train = reformat(X_train, y_train)
X_valid, y_valid = reformat(X_valid, y_valid)
X_test, y_test = reformat(X_test, y_test)
y_train = np_utils.to_categorical(y_train, num_labels)
y_test = np_utils.to_categorical(y_test, num_labels)
y_valid = np_utils.to_categorical(y_valid, num_labels)
print('Training set', X_train.shape, y_train.shape)
print('Validation set', X_valid.shape, y_valid.shape)
#print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (12019, 2500), (12019, 10))
('Validation set', (3005, 2500), (3005, 10))


In [7]:
from keras.models import Sequential
from keras.layers import Dense
#import numpy
# fix random seed for reproducibility
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(output_dim=64, input_dim=2500))
model.add(Activation("relu"))
model.add(Dense(output_dim=10))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [27]:
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
csv_logger = keras.callbacks.CSVLogger('/home/mohammad/training.log')
model.fit(X_train, y_train, nb_epoch=10, batch_size=32,verbose=1,
          show_accuracy=True, callbacks=[csv_logger],
              validation_data=(X_valid, y_valid))

Train on 12019 samples, validate on 3005 samples
Epoch 1/10
12019/12019 [==============================] - 1s - loss: 0.3102 - acc: 0.9388 - val_loss: 0.3982 - val_acc: 0.9002
Epoch 2/10
12019/12019 [==============================] - 1s - loss: 0.2740 - acc: 0.9468 - val_loss: 0.2921 - val_acc: 0.9371
Epoch 3/10
12019/12019 [==============================] - 1s - loss: 0.2460 - acc: 0.9537 - val_loss: 0.2565 - val_acc: 0.9517
Epoch 4/10
12019/12019 [==============================] - 1s - loss: 0.2199 - acc: 0.9591 - val_loss: 0.2478 - val_acc: 0.9484
Epoch 5/10
12019/12019 [==============================] - 1s - loss: 0.1996 - acc: 0.9634 - val_loss: 0.2264 - val_acc: 0.9488
Epoch 6/10
12019/12019 [==============================] - 1s - loss: 0.1840 - acc: 0.9668 - val_loss: 0.2322 - val_acc: 0.9494
Epoch 7/10
12019/12019 [==============================] - 1s - loss: 0.1659 - acc: 0.9712 - val_loss: 0.1951 - val_acc: 0.9624
Epoch 8/10
12019/12019 [==============================] - 1s -

In [9]:
from sklearn.metrics import log_loss
predictions_valid = model.predict(X_valid, batch_size=128, verbose=1)
score = log_loss(y_valid, predictions_valid)
print('Score log_loss: ', score)

2944/3005 [============================>.] - ETA: 0s('Score log_loss: ', 0.40482608707961254)


In [10]:
classes = model.predict_classes(X_test, batch_size=32)
proba = model.predict_proba(X_test, batch_size=32)
predictions_test = model.predict(X_test, batch_size=128, verbose=1)
score = log_loss(y_test, predictions_test)
print('Score log_loss: ', score)

7040/7400 [===========================>..] - ETA: 0s('Score log_loss: ', 0.41561942186004425)


In [28]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

('Test score:', 0.19712647291454108)
('Test accuracy:', 0.95297297297297301)


In [12]:
y_test[1]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.])

In [13]:
print log_loss(y_test[0],predictions_test[0])


0.0144391550362


In [14]:
np.round(predictions_test[0],1)

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.89999998,  0.        ,  0.        ,  0.        ,  0.        ], dtype=float32)

In [30]:
#loss_history = keras.callbacks.History
a = np.array(['score','accuracy'])
b = np.column_stack((a,score))

In [33]:
np.savetxt('/home/mohammad/test.txt', b, delimiter=",", fmt="%s") 